In [18]:
%matplotlib inline
# not sure if necessary

In [19]:
import finetune
import importlib
import torch

In [20]:
# run this cell when you change stuff in the finetune module, and don't want to restart the notebook's kernel.
importlib.reload(finetune)
finetune.structure_checkpoints()

cannot safely create ../../data\finetuned_models
cannot safely create ../../data\finetuned_models\StanfordCars
cannot safely create ../../data\finetuned_models\StanfordCars\resnet
cannot safely create ../../data\finetuned_models\StanfordCars\alexnet
cannot safely create ../../data\finetuned_models\StanfordCars\vgg
cannot safely create ../../data\finetuned_models\StanfordCars\squeezenet
cannot safely create ../../data\finetuned_models\StanfordCars\densenet
cannot safely create ../../data\finetuned_models\StanfordCars\inception
cannot safely create ../../data\finetuned_models\FGVC-Aircraft
cannot safely create ../../data\finetuned_models\FGVC-Aircraft\resnet
cannot safely create ../../data\finetuned_models\FGVC-Aircraft\alexnet
cannot safely create ../../data\finetuned_models\FGVC-Aircraft\vgg
cannot safely create ../../data\finetuned_models\FGVC-Aircraft\squeezenet
cannot safely create ../../data\finetuned_models\FGVC-Aircraft\densenet
cannot safely create ../../data\finetuned_models\FG

In [21]:
finetune.CLASSIFIER_NAME = 'resnet'
finetune.NUM_EPOCHS = 15
finetune.FEATURE_EXTRACT = False
finetune.DATASET = 'StanfordCars'
finetune.print_hypers() # shows params

# Initialize the model for this run
model_ft = finetune.initialize_model(_verbose=False)

classifier input size: None
batch size: 8
num epochs: 15
feature extract: False


In [22]:
model, hist, state = finetune.finetune_model(model_ft)

  0%|          | 0/1018 [00:00<?, ?it/s]

Epoch 0/14
----------


  0%|          | 0/503 [00:00<?, ?it/s]

train Loss: 4.9115 Acc: 0.0426


  0%|          | 0/1018 [00:00<?, ?it/s]

...

val Loss: 0.7481 Acc: 0.7908

Epoch 14/14
----------


  0%|          | 0/503 [00:00<?, ?it/s]

train Loss: 1.0046 Acc: 0.7517


100%|██████████| 503/503 [00:09<00:00, 51.02it/s]

val Loss: 0.7788 Acc: 0.7808

Training complete in 13m 14s
Best val Acc: 0.790796


In [23]:
model_save_path = finetune.format_model_path(finetune.CLASSIFIER_NAME,
                                             finetune.DATASET,
                                             state['epochs'])

finetune.save_model(state, model_save_path)

saving model to ../../data\finetuned_models\StanfordCars\resnet\resnet_StanfordCars_E15.pth


In [24]:
# Loading certain model
model_props = {
    'name': 'resnet',
    'dataset': 'StanfordCars',
    'epochs': 15
}
model_load_path = finetune.format_model_path(model_props['name'],
                                             model_props['dataset'],
                                             model_props['epochs'])
,
state = torch.load(model_load_path)
new_model = finetune.get_model_architecture(model_props['name'])
new_model.load_state_dict(state['model_state_dict'])
optimizer_state_dict = state['optimizer_state_dict']

In [25]:
finetune.NUM_EPOCHS = 15
finetune.FEATURE_EXTRACT = False
finetune.DATASET = 'StanfordCars'

new_model, hist2, state2 = finetune.finetune_model(new_model,optimizer_state_dict, is_retrain=model_props['epochs'])

  0%|          | 0/1018 [00:00<?, ?it/s]

Epoch 0/14
----------


  0%|          | 0/503 [00:00<?, ?it/s]

train Loss: 1.0174 Acc: 0.7417


  0%|          | 0/1018 [00:00<?, ?it/s]

...

val Loss: 0.6626 Acc: 0.8159

Epoch 14/14
----------


  0%|          | 0/503 [00:00<?, ?it/s]

train Loss: 0.7062 Acc: 0.8211


100%|██████████| 503/503 [00:09<00:00, 52.31it/s]

val Loss: 0.6671 Acc: 0.8122

Training complete in 13m 9s
Best val Acc: 0.817662


In [26]:
model_save_path = finetune.format_model_path(finetune.CLASSIFIER_NAME,
                                             finetune.DATASET,
                                             state2['epochs'])

finetune.save_model(state2, model_save_path)

saving model to ../../data\finetuned_models\StanfordCars\resnet\resnet_StanfordCars_E30.pth


In [ ]:
finetune.NUM_CLASSES = 196
finetune.CLASSIFIER_NAME = 'resnet'
finetune.NUM_EPOCHS = 15
finetune.FEATURE_EXTRACT = True
finetune.print_hypers() # shows params
# Initialize the model for this run
model_ft = finetune.initialize_model()


# Print the model we just instantiated
print(model_ft)
finetune.finetune_model(model_ft)

In [ ]:
finetune.NUM_CLASSES = 196
finetune.CLASSIFIER_NAME = 'resnet'
finetune.NUM_EPOCHS = 30
finetune.FEATURE_EXTRACT = True
finetune.print_hypers() # shows params
# Initialize the model for this run
model_ft = finetune.initialize_model()


# Print the model we just instantiated
print(model_ft)
finetune.finetune_model(model_ft)